In [1]:
import numpy as np
import torch
import torchvision
import torch.nn.functional as F
from torch.utils.data import DataLoader
import preprocessing
# import training
import importlib
import pandas as pd

importlib.reload(preprocessing)
# importlib.reload(training)

<module 'preprocessing' from 'C:\\Users\\dbsrh\\OneDrive\\Desktop\\Pytorch\\kaggle_digit_recognizer\\preprocessing.py'>

In [2]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=3)
        self.conv2 = torch.nn.Conv2d(10, 15, kernel_size=3)
        self.fc1 = torch.nn.Linear(375, 50)
        self.fc2 = torch.nn.Linear(50, 10)
        
    def forward(self, x):
        # 28 X 28 --> 26 X 26 --> 13 X 13
        x = self.conv1(x)
        x = F.relu(F.max_pool2d(x, 2))
        # 13 X 13 --> 11 X 11 --> 5 X 5
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        # 5 X 5 X 15
        x = x.view(-1, 375)
        x = self.fc1(x)
        x = F.dropout(x, p=0.3, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
    
def training(model, datasets, optimizer, scheduler, epochs=10):
    phases = ['valid']
    
    training_loss = []
    validation_loss = []
    
    for epoch in range(epochs):                
        print(f'-------------- Epoch {epoch} --------------')
        epoch_corrects = 0
        epoch_losses = 0.0
        
        for phase in phases:
            if phase == 'train':
                model.train()
                optimizer.zero_grad()
                
            if phase == 'valid':
                model.eval()
                
            for data in datasets[phase]:
                inputs, labels = data
                if torch.cuda.is_available():
                    inputs, labels = inputs.cuda(), labels.cuda()
            
                print(inputs.shape)
                output = model(inputs)
                loss = F.nll_loss(output, labels, reduction='none').data[0]
                print(output)
                print('-----------------')
                print(labels)
                print('-----------------')
                print(loss)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
        
        scheduler.step()

In [12]:
# Define CNN model
model_ft = Model()

if torch.cuda.is_available():
    model_ft.cuda()

# Define image data transforms
transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.Resize((28, 28)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5], [0.5])
])


# Load data
valid_fraction = 0.1

csv_file = pd.read_csv('./digit-recognizer/train.csv')
csv_file = csv_file.to_numpy().astype('uint8')

boundary = int(len(csv_file) * (1 - 0.1))

train_data = preprocessing.DigitDataset(csv_file[:boundary], transforms, phase='training')
valid_data = preprocessing.DigitDataset(csv_file[boundary:], transforms, phase='training')

train_data_loader = DataLoader(
    train_data,
    batch_size = 32,
    shuffle = True,
    num_workers = 3
)

valid_data_loader = DataLoader(
    valid_data,
    batch_size = 32,
    shuffle = True,
    num_workers = 3
)

datasets = {'train' : train_data_loader, 'valid' : valid_data_loader}
datasets_size = {'train' : len(train_data_loader.dataset), 'valid' : len(valid_data_loader.dataset)}

In [13]:
optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=0.001)
scheduler_ft = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=10)

In [14]:
training(model_ft, datasets, optimizer_ft, scheduler_ft)

-------------- Epoch 0 --------------
torch.Size([32, 1, 28, 28])


E:\Users\dbsrh\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


RuntimeError: Expected object of scalar type Long but got scalar type Byte for argument #2 'target' in call to _thnn_nll_loss_forward

In [ ]:
for data in valid_data_loader:    
    inputs, labels = data
    print(inputs)
#     print(valid_data_loader.dataset)
    break
#     tmp_shape = [ str(element) for element in list(tmp_shape)]
#     tmp_str = ','.join(tmp_shape)
#     if tmp_str in check_dict:
#         check_dict[tmp_str] += 1
#     else:
#         check_dict[tmp_str] = 1

{'28,28': 4200}
